In [10]:
import pandas as pd

# --- NOMS DE COLONNES BASÉS SUR VOTRE EXEMPLE ---
FILE_NAME = 'data/comptages-routiers-permanents_convention.csv' 
COL_DATETIME = 'Date et heure de comptage'  # Nom exact
COL_DEBIT = 'Débit horaire'                 # Nom exact
COL_TAUX = 'Taux d\'occupation'             # Nom exact
# ------------------------------------------------

# Chargement du fichier CSV (le séparateur est supposé être ';')
df = pd.read_csv(FILE_NAME, sep=';')

# Colonnes à garder pour le traitement minimaliste
COLUMNS_TO_KEEP = [COL_DATETIME, COL_DEBIT, COL_TAUX] 

# Sélection des colonnes utiles
df_selected = df[COLUMNS_TO_KEEP].copy()

print(f"Fichier chargé. Total d'enregistrements initiaux : {len(df)}")
print(f"Colonnes sélectionnées : {list(df_selected.columns)}")
print("Aperçu des données sélectionnées:")
print(df_selected.head())

Fichier chargé. Total d'enregistrements initiaux : 8627
Colonnes sélectionnées : ['Date et heure de comptage', 'Débit horaire', "Taux d'occupation"]
Aperçu des données sélectionnées:
   Date et heure de comptage  Débit horaire  Taux d'occupation
0  2025-09-02T11:00:00+02:00          462.0            4.20056
1  2025-10-09T11:00:00+02:00          514.0            3.86722
2  2025-09-02T12:00:00+02:00          482.0            3.24945
3  2025-09-02T13:00:00+02:00          563.0            4.57445
4  2024-10-04T12:00:00+02:00            NaN                NaN


In [12]:
# --- NOMS DE COLONNES ET FICHIERS ---
EXTERNAL_FILE_NAME = 'data/external_features_final.csv'
EXTERNAL_COL_TIME = 'time'
MAIN_COL_TIME = 'Date et heure de comptage' 
COL_DEBIT = 'Débit horaire'
COL_TAUX = 'Taux d\'occupation' 
# ------------------------------------

# 1. Chargement du fichier des features externes (séparateur : ,)
df_external = pd.read_csv(EXTERNAL_FILE_NAME, sep=',')

# 2. Préparation de la clé de fusion : conversion en datetime pour les deux DataFrames
df_selected['datetime_merge_key'] = pd.to_datetime(df_selected[MAIN_COL_TIME], utc=True)
df_external['datetime_merge_key'] = pd.to_datetime(df_external[EXTERNAL_COL_TIME], utc=True)

# 3. Suppression des colonnes redondantes dans le fichier externe
df_external = df_external.drop(columns=[EXTERNAL_COL_TIME, 'date', 'hour'], errors='ignore')

# 4. Fusion des DataFrames sur la clé temporelle
df_selected_merged = pd.merge(
    df_selected,
    df_external,
    on='datetime_merge_key',
    how='left'
)

# 5. Remplacement du DataFrame de travail pour la suite du nettoyage (Bloc 2)
df_selected = df_selected_merged.drop(columns=['datetime_merge_key'])

print("\n--- Fusion des Features Externes Terminée ---")
print(f"Nouvelles dimensions du DataFrame : {df_selected.shape}")

# CORRECTION DE LA SYNTAXE ICI
print(f"Nouvelles colonnes ajoutées : {df_selected.columns.difference([MAIN_COL_TIME, COL_DEBIT, COL_TAUX]).tolist()}")
print(df_selected.head())


--- Fusion des Features Externes Terminée ---
Nouvelles dimensions du DataFrame : (8627, 13)
Nouvelles colonnes ajoutées : ['cloud_cover', 'has_event', 'has_event_champs', 'has_event_convention', 'has_event_stsperes', 'is_holiday', 'is_vacation', 'precipitation', 'temperature_2m', 'wind_speed_10m']
   Date et heure de comptage  Débit horaire  Taux d'occupation  \
0  2025-09-02T11:00:00+02:00          462.0            4.20056   
1  2025-10-09T11:00:00+02:00          514.0            3.86722   
2  2025-09-02T12:00:00+02:00          482.0            3.24945   
3  2025-09-02T13:00:00+02:00          563.0            4.57445   
4  2024-10-04T12:00:00+02:00            NaN                NaN   

   temperature_2m  precipitation  cloud_cover  wind_speed_10m  is_holiday  \
0            15.4            0.0          100            18.2         0.0   
1            14.0            0.0           36             6.4         0.0   
2            16.3            0.0          100            19.4         0

In [14]:
df_selected

,Date et heure de comptage,Débit horaire,Taux d'occupation,temperature_2m,precipitation,cloud_cover,wind_speed_10m,is_holiday,is_vacation,has_event,has_event_champs,has_event_stsperes,has_event_convention
0,2025-09-02T11:00:00+02:00,462.0,4.20056,15.4,0.0,100,18.2,0.0,False,0,1,1,1
1,2025-10-09T11:00:00+02:00,514.0,3.86722,14.0,0.0,36,6.4,0.0,False,0,1,1,1
2,2025-09-02T12:00:00+02:00,482.0,3.24945,16.3,0.0,100,19.4,0.0,False,0,1,1,1
3,2025-09-02T13:00:00+02:00,563.0,4.57445,17.5,0.0,100,17.2,0.0,False,0,1,1,1
4,2024-10-04T12:00:00+02:00,NaN,NaN,10.5,0.0,0,7.7,0.0,False,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8622,2024-10-30T05:00:00+01:00,NaN,NaN,11.3,0.0,100,1.9,0.0,True,0,0,1,1
8623,2024-10-30T04:00:00+01:00,NaN,NaN,10.8,0.0,100,2.8,0.0,True,0,0,1,1
8624,2024-10-30T01:00:00+01:00,NaN,NaN,10.3,0.0,100,5.6,0.0,True,0,0,1,1
8625,2024-11-01T03:00:00+01:00,NaN,NaN,10.4,0.0,100,6.6,1.0,True,0,0,1,1
